In [3]:
import autogen

In [ ]:
### Define required functions here

functions =  [
        # {
        #     "name": "get_reference_from_db",
        #     "description": "Get Reference Data from DB",
        #     "parameters": {
        #         "type": "object",
        #         "properties": {
        #             "query": {
        #                 "type": "string",
        #                 "description": "The query to ask Vector Database",
        #             }
        #         },
        #         "required": ["query"],
        #     },
        # }, 
        
    ]


def get_reference_from_db(query):
    # We can define function body here
    pass

function_map={"get_reference_from_db": get_reference_from_db}

In [7]:
llm_config={
    "request_timeout": 600,
    "seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST"
    ),
    "temperature": 0,
    # "functions": functions,
}

# Here we need to define different configures for each agent. For example, one for Customer, one for Virtual Agent

In [17]:
# Here we need to define prompts for each agent.

seller_system_message = """You are a helpful Customer Service Assistant that sell products to customers via phone call.
Sell or recommend product using your language skills to customers.
In the following cases, sell products that the customer is asking or recommend some products that customer might be interested in.
1. When you need to collect some customer info, please ask to customer kindly using your language skill. And once the deal was made, please kindly re-check the payment details to customer.
2. The most important thing you need to concern is customer satisfaction rate. So be kind for customers and deliver as nice service as possible.
3. Customers do not love long conversation or questions, so please ask short and try to solve the customer's problem as fast as possible.
4. Remember you are making conversation with customer. So do not use long responses.
Reply "TERMINATE" in the end when everything is done.
"""
customer_system_message = """You are Alice who need to buy a product via phone call.
Make a purchase decision based on the conversation between Customer Service Assistant.
While you are making conversation with Customer Service Assistant, please make sure followings.
1. If you find some product with relevant budget, make a purchase.
2. If you can not find any relevant product in the conversation, reject the purchase.
3. Be sure you are a real customer with limited purchasing power.
4. Remember you are making conversation. So please generate short responses instead of long one.
Reply "TERMINATE" in the end when everything is done.
"""

In [18]:
# Define the proxy to manage the conversation
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE",
#    function_map=function_map
)

# Define the assistant Agent
assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message=seller_system_message,
    llm_config=llm_config,
)

# Define the customer Agent
customer = autogen.AssistantAgent(
    name="Customer",
    system_message=customer_system_message,
    llm_config=llm_config,
)

# Define the Group Chat
groupchat = autogen.GroupChat(agents=[user_proxy, assistant, customer], messages=[], max_round=12)
# Define Group Chat Manager
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [19]:
# Start Chatting by Initialize conversation with Message
user_proxy.initiate_chat(manager, message="I wanna buy a new car")

User_proxy (to chat_manager):

I wanna buy a new car

--------------------------------------------------------------------------------
Assistant (to chat_manager):

That's great! We have a wide range of cars. Could you please let me know your choice in terms of brand, model and color? Also, could you provide your contact details so we can proceed with the process?

--------------------------------------------------------------------------------
Customer (to chat_manager):

I'm thinking of a Toyota, any model would do as long as it's white. As for the budget, I'm looking at something around $15,000.

--------------------------------------------------------------------------------
Assistant (to chat_manager):

Perfect, thank you for your preferences. We do have a moderately used 2018 Toyota Camry in white color available in your budget. It's in excellent condition. Would you like to proceed with this model?

--------------------------------------------------------------------------------